In [1]:
import geopandas as gpd
import pandas as pd
from google_geocoding import *
from feature_dev import *
from utils import *

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_rows', None)

# df = develop_features()

In [2]:
# # Get station points
# gdf = load_station_points()

# # Extract lats and lons
# gdf['lat'] = gdf.geometry.y
# gdf['lon'] = gdf.geometry.x

# # Make cartesian product of stations (all OD pairs)
# df = pd.merge(gdf[['station_name','lat','lon']], gdf[['station_name','lat','lon']], how='cross', suffixes=('_o', '_d'))

# # Remove diagonal
# df = df[df.station_name_o != df.station_name_d]

# # Split into 10 randomly-assigned groups
# df = assign_random_groups(df, 10)
# dfs = split_dataframe_by_id(df, 'group')

# # Pickle group dataframes
# with open('group_dataframes.pickle', 'wb') as handle:
#     pickle.dump(dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
# Load group dataframes
with open('group_dataframes.pickle', 'rb') as handle:
    dfs = pickle.load(handle)

In [5]:
# modes = ['driving']
# modes = [('transit','bus')]
# modes = [('transit','subway')]

modes = [('transit','bus'), ('transit','subway')]

group = 1
df = dfs[group]

name = '_'.join(flatten(modes))
name = f'{name}_group{group}'

dict = query_routes(df, configs.google_api_key, pickle_path=f'{name}_routes.pickle', modes=modes)
results_df = summarize_routes(dict)
results_df.to_parquet(f'{name}_summaries.parquet')

Completed 50 routes
Completed 100 routes
Completed 150 routes
Completed 200 routes
Completed 250 routes
Completed 300 routes
Completed 350 routes
Completed 400 routes
Completed 450 routes
Completed 500 routes
Completed 550 routes
Completed 600 routes
Completed 650 routes
Completed 700 routes
Completed 750 routes
Completed 800 routes
Completed 850 routes
Completed 900 routes
Completed 950 routes
Completed 1000 routes


In [8]:
dt_df = pd.read_parquet('transit_bus_transit_subway_group0_summaries.parquet')

In [9]:
len(dt_df)

1001

In [ ]:
df = pd.concat([df.reset_index(drop=True), dt_df[['google_driving_minutes','google_driving_miles']]], axis=1)

In [ ]:
df.to_csv('google_driving_summaries.csv')

In [ ]:
dict = get_route_info(
    orig_lat=df.iloc[0].lat_o, 
    orig_lon=df.iloc[0].lon_o, 
    dest_lat=df.iloc[0].lat_d, 
    dest_lon=df.iloc[0].lon_d, 
    api_key=configs.google_api_key, 
    mode='transit', 
    transit_mode='bus'
)

In [ ]:
dict.headers

In [ ]:
dir(dict)